In [1]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, LeakyReLU, Input
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from scikeras.wrappers import KerasClassifier
from sklearn.base import ClassifierMixin
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

class SklearnKerasClassifier(KerasClassifier, ClassifierMixin):
    def __init__(self, model=None, **kwargs):
        super().__init__(model=model, **kwargs)
    @property
    def _tags(self):
        return self.model._tags if hasattr(self.model, "_tags") else {"binary_only": True}

def create_custom_mlp(hidden_layer_configs=(50, 50),
                        dropout_rate=0.3,
                        activation='leaky_relu',
                        learning_rate=0.001,
                        meta=None):
    if meta is None:
        raise ValueError("meta parametresi eksik")
        
    n_features_in = meta["n_features_in_"]
    layer1_neurons, layer2_neurons = hidden_layer_configs
    model = Sequential(name="custom_MLP_trial")
    model.add(Input(shape=(n_features_in,)))
    
    model.add(Dense(layer1_neurons))
    model.add(BatchNormalization())
    if activation == 'leaky_relu': model.add(LeakyReLU(alpha=0.1))
    else: model.add(tf.keras.layers.ReLU())
    model.add(Dropout(dropout_rate))
    
    if layer2_neurons is not None and layer2_neurons > 0:
        model.add(Dense(layer2_neurons))
        model.add(BatchNormalization())
        if activation == 'leaky_relu':
            model.add(LeakyReLU(alpha=0.1))
        else:
            model.add(tf.keras.layers.ReLU())
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

X, y = make_classification(
    n_samples=500, n_features=100, n_informative=50,
    n_redundant=20, n_classes=2, random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Veri seti oluşturuldu. Eğitim seti boyutu: {X_train.shape}, Test seti boyutu: {X_test.shape}")

keras_estimator = SklearnKerasClassifier(
    model=create_custom_mlp,
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam,
    optimizer__learning_rate=0.001,
    epochs=20,
    batch_size=32,
    verbose=0,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, restore_best_weights=True)]
)

param_grid = {
    'model__hidden_layer_configs': [(64, 32), (128, 64), (100, None)],
    'model__dropout_rate': [0.2, 0.4],
    'model__activation': ['relu', 'leaky_relu'],
    'optimizer__learning_rate': [0.001, 0.005]
}

cv_strategy = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

search_cv = RandomizedSearchCV(
    estimator=keras_estimator,
    param_distributions=param_grid,
    n_iter=4,
    cv=cv_strategy,
    scoring='f1',
    verbose=2, # output
    random_state=42
)

try:
    search_cv.fit(X_train, y_train, validation_split=0.2)
    print(f"En iyi F1 skoru (CV): {search_cv.best_score_:.4f}")
    print("En iyi parametreler:")
    print(search_cv.best_params_)
    best_model = search_cv.best_estimator_
    test_score = best_model.score(X_test, y_test)
    print(f"\naccuracy: {test_score:.4f}")

except Exception as e:
    print(f"\hata: {e}")
    import traceback
    traceback.print_exc()

Veri seti oluşturuldu. Eğitim seti boyutu: (400, 100), Test seti boyutu: (100, 100)
Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] END model__activation=relu, model__dropout_rate=0.4, model__hidden_layer_configs=(128, 64), optimizer__learning_rate=0.001; total time=   2.9s
[CV] END model__activation=relu, model__dropout_rate=0.4, model__hidden_layer_configs=(128, 64), optimizer__learning_rate=0.001; total time=   2.9s
[CV] END model__activation=relu, model__dropout_rate=0.4, model__hidden_layer_configs=(128, 64), optimizer__learning_rate=0.001; total time=   2.8s


C:\Users\BerenÜnveren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


[CV] END model__activation=leaky_relu, model__dropout_rate=0.2, model__hidden_layer_configs=(100, None), optimizer__learning_rate=0.001; total time=   2.0s


C:\Users\BerenÜnveren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


[CV] END model__activation=leaky_relu, model__dropout_rate=0.2, model__hidden_layer_configs=(100, None), optimizer__learning_rate=0.001; total time=   1.9s


...

[CV] END model__activation=leaky_relu, model__dropout_rate=0.4, model__hidden_layer_configs=(64, 32), optimizer__learning_rate=0.001; total time=   2.4s


C:\Users\BerenÜnveren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


[CV] END model__activation=leaky_relu, model__dropout_rate=0.4, model__hidden_layer_configs=(64, 32), optimizer__learning_rate=0.001; total time=   2.5s


C:\Users\BerenÜnveren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


En iyi F1 skoru (CV): 0.8344
En iyi parametreler:
{'optimizer__learning_rate': 0.001, 'model__hidden_layer_configs': (100, None), 'model__dropout_rate': 0.2, 'model__activation': 'leaky_relu'}

accuracy: 0.7600
